In [ ]:
import torch.nn as nn
import torch
# Define the model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, hn = self.rnn(x, h0.detach())
        out = self.fc(out) 
        return out


In [ ]:
#Define architecture
input_dim = 5
hidden_dim = 15
layer_dim = 1 
output_dim = 3

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

In [ ]:
import xlrd
import numpy as np
from sklearn.preprocessing import StandardScaler

#load trained model 
model.load_state_dict(torch.load("C:/Users/Gabriel/Documents/Python/RNN1.pt"))
model.eval()

#load dataset used for standardisation
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data4 = [[sheet.cell_value(r, c)
         for c in range(1,6)] for r in range(2,74)]
data5 = [[sheet.cell_value(r, c)
         for c in range(6,9)] for r in range(2,74)]

scaler = StandardScaler()
scaler2 = StandardScaler()


In [ ]:
#load dataset 7
book = xlrd.open_workbook('Experimental Data edited.xlsx')
sheet = book.sheet_by_name('Sheet2')
#choose the number of timesteps the model will 'see' before predicting the rest 
points_given=1
data = [[sheet.cell_value(r, c)
         for c in range(1,6)] for r in range(114,114+points_given)]
data3 = [[sheet.cell_value(r, c)
         for c in range(1,4)] for r in range(115,127)]
data2 = [[sheet.cell_value(r, c)
         for c in range(1,6)] for r in range(114,126)]
actual_results = torch.tensor(data3)

scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(data4)
scaler2.fit(data5)

#sequence the data
test_data = torch.tensor(data)
test_data_all = torch.tensor(data2)
sequenced_test_data_all = test_data_all.view(1,12,5)
sequenced_test_data = test_data.view(1,points_given,5)


for i in range (12-points_given):
    #standardise the data
    unsequenced_test_data = sequenced_test_data.view(i+points_given,5)
    scaled_test_data = torch.tensor(scaler.transform(unsequenced_test_data))
    sequenced_scaled_test_data = scaled_test_data.view(1,i+points_given,5)
    with torch.no_grad():
        #predict the rate of change
         data_pred = model(sequenced_scaled_test_data.float())
    predictions = np.array(data_pred)
    #destandardise the rate of change 
    descaled_predictions = scaler2.inverse_transform(predictions)
    usable_predictions = torch.tensor(descaled_predictions)
    #add the rate of change to the previous timestep for prediction of the next timestep
    true_predictions = (sequenced_test_data[:,(i+points_given-1):(i+points_given),0:3] +
    (usable_predictions[:,(i+points_given-1):(i+points_given),:]*12))
    tensor_predictions = torch.tensor(true_predictions.float())
    #append prediction of the next timestep to the bottom of the current sequence
    prediction_added = torch.cat((sequenced_test_data.float()[:,:,0:3],
                              tensor_predictions[:,:,:]),1)
    #add light intensity and nitrate inflow conc data to the new timestep
    sequenced_test_data = torch.cat((prediction_added,
                               sequenced_test_data_all.float()
                                     [:,0:(points_given+1)+i,3:5]),2)
    final_predictions = sequenced_test_data

x_plot = final_predictions.view(12,5)
   

In [ ]:
#calculate MAPE and generate graphs
def MAPE(y_true, y_pred):
    MAPE_Total=0
    for i in range(len(y_pred)):
        MAPE = torch.abs((y_true[i:1+i,:]-y_pred[i:1+i,:])/y_true[i:1+i,:])
        MAPE_Total += (MAPE/len(y_pred))*100
    return MAPE_Total

import numpy as np
a = np.linspace(12,144,12)
v = torch.tensor(data3)
import matplotlib.pyplot as plt 
y1 = x_plot[:, 0:1] 
x1 = a
plt.plot(x1, y1,'o' ,label = "predicted") 

y2 = v[:, 0:1] 
x2 = a
plt.plot(x2, y2, 'o',label = "experimental") 
plt.xlabel('time (h)')
plt.ylabel('Biomass Conc (g/L)')
loss = MAPE(v[:, 0:1],x_plot[:, 0:1])
plt.title(('MAPE = {:.2f}'.format(loss.item())))
plt.grid(b=True,which='major', axis='both')
plt.legend()
plt.show() 


y1 = x_plot[:, 1:2] 
x1 = a
plt.plot(x1, y1,'o',label = "predicted") 

y2 = v[:, 1:2] 
x2 = a
plt.plot(x2, y2,'o',label = "experimental") 
plt.xlabel('time (h)')
plt.ylabel('Nitrate Conc (mg/L)')
loss = MAPE(v[:, 1:2],x_plot[:, 1:2])
plt.title(('MAPE = {:.2f}'.format(loss.item())))
plt.grid(b=True,which='major', axis='both')
plt.legend()
plt.show() 

y1 = x_plot[:, 2:3] 
x1 = a
plt.plot(x1, y1,'o',label = "predicted") 

y2 = v[:, 2:3] 
x2 = a
plt.plot(x2, y2,'o',label = "experimental") 
plt.xlabel('time (h)')
plt.ylabel('Lutein Conc (mg/L)')
loss = MAPE(v[:, 2:3],x_plot[:, 2:3])
plt.title(('MAPE = {:.2f}'.format(loss.item())))
plt.grid(b=True,which='major', axis='both')
plt.legend()
plt.show() 



In [ ]:
#export results to excel
import pandas as pd
b=np.array(v)
c=np.array(x_plot)
book = xlrd.open_workbook('tested data.xlsx')
sheet = book.sheet_by_name('Sheet1')
writer = pd.ExcelWriter('tested data.xlsx', engine='xlsxwriter')
df1 = pd.DataFrame(b)
df1.to_excel(writer, header=True, index=True)
df2 = pd.DataFrame(c)
df2.to_excel(writer, startrow=13, header=True, index=True)
writer.save()